In [1]:
# ---
# FEATURE EXTRACTION – CNN (VGG16)
# Senior Engineer Notes:
# - Extract deep CNN features from each frame
# - Saveas numpy arrays for LSTM model
# -----------------------------------------------------

import numpy as np
import cv2
import os
from pathlib import Path
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model

# Detect project root (consistent with previous notebooks)
repo_root = Path.cwd().parent if "notebooks" in str(Path.cwd()) else Path.cwd()

processed_dir = repo_root / "data" / "processed"
features_dir = repo_root / "data" / "features"

features_dir.mkdir(parents=True, exist_ok=True)

print("Processed frames:", processed_dir)
print("Feature output:", features_dir)


Processed frames: D:\VidSense\data\processed
Feature output: D:\VidSense\data\features


In [2]:
# -----------------------------------------------------
# Load pretrained VGG16model
# -----------------------------------------------------

# base VGG16
base_model = VGG16(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

print("VGG16 model loaded. Output feature dim:", model.output_shape)


VGG16 model loaded. Output feature dim: (None, 512)


In [3]:
# -----------------------------------------------------
# Extract CNN features from a single frame
# ---------------------------------------------------

def extract_feature_from_frame(frame):
    frame_resized = cv2.resize(frame, (224, 224))
    frame_array = np.expand_dims(frame_resized.astype("float32"), axis=0)
    frame_processed = preprocess_input(frame_array)
    feature = model.predict(frame_processed, verbose=0)
    return feature.flatten()  # shape: (512,--)


In [4]:
# -----------------------------------------------------
# Loop through processed frames folder and extract features
# -----------------------------------------------------

labels = [d.name for d in processed_dir.iterdir() if d.is_dir()]
print("Detected labels:", labels)

for label in labels:
    label_features_dir = features_dir / label
    label_features_dir.mkdir(exist_ok=True)

    label_frame_dir = processed_dir / label
    videos = [v for v in label_frame_dir.iterdir() if v.is_dir()]

    print(f"\nProcessing label: {label} | {len(videos)} videos")

    for vid_folder in videos:
        frames = sorted(list(vid_folder.glob("*.jpg")))

        if len(frames) == 0:
            continue

        features_list = []

        for frame_path in frames:
            frame = cv2.imread(str(frame_path))
            feat = extract_feature_from_frame(frame)
            features_list.append(feat)

        video_features = np.array(features_list)  # shape: (num_frames, 512)

        out_path = label_features_dir / f"{vid_folder.name}.npy"
        np.save(out_path, video_features)

        print(f"✔ Saved features for: {vid_folder.name} → {video_features.shape}")


Detected labels: ['falling', 'running', 'sitting', 'walking']

Processing label: falling | 3 videos
✔ Saved features for: falling_demo_1 → (12, 512)
✔ Saved features for: falling_demo_2 → (12, 512)
✔ Saved features for: falling_demo_3 → (12, 512)

Processing label: running | 3 videos
✔ Saved features for: running_demo_1 → (12, 512)
✔ Saved features for: running_demo_2 → (12, 512)
✔ Saved features for: running_demo_3 → (12, 512)

Processing label: sitting | 3 videos
✔ Saved features for: sitting_demo_1 → (12, 512)
✔ Saved features for: sitting_demo_2 → (12, 512)
✔ Saved features for: sitting_demo_3 → (12, 512)

Processing label: walking | 3 videos
✔ Saved features for: walking_demo_1 → (12, 512)
✔ Saved features for: walking_demo_2 → (12, 512)
✔ Saved features for: walking_demo_3 → (12, 512)


In [6]:
print("\n===== SUMMARY =======")

for label in labels:
    label_dir = features_dir / label
    if not label_dir.exists():
        continue

    print(f"\nLabel: {label}")
    for npy_file in label_dir.glob("*.npy"):
        data = np.load(npy_file)
        print(f"  {npy_file.name}: {data.shape}")



===== SUMMARY =======

Label: falling
  falling_demo_1.npy: (12, 512)
  falling_demo_2.npy: (12, 512)
  falling_demo_3.npy: (12, 512)

Label: running
  running_demo_1.npy: (12, 512)
  running_demo_2.npy: (12, 512)
  running_demo_3.npy: (12, 512)

Label: sitting
  sitting_demo_1.npy: (12, 512)
  sitting_demo_2.npy: (12, 512)
  sitting_demo_3.npy: (12, 512)

Label: walking
  walking_demo_1.npy: (12, 512)
  walking_demo_2.npy: (12, 512)
  walking_demo_3.npy: (12, 512)
